In [1]:
import nest_asyncio
from pydantic import BaseModel, Field
from crewai.flow import Flow, start, listen
from litellm import completion

In [2]:
# apply nest_asyncio to allow nested event loops in jupyter
nest_asyncio.apply()

In [3]:
class AISummaryState(BaseModel):
    topic: str = Field(default="Agentic AI in Health care", descrtion="The topic for summary")
    headlines: str=""
    summary: str=""

In [4]:
class AISummaryFlow(Flow[AISummaryState]):
    @start()
    def fetch_info(self):
        print(f"fetch_info: self.state.topic = {self.state.topic}")
        response = completion(
            model="gpt-4o-mini",
            message=[{"role":"user", "content":f"Get top 3 topics on {self.state.topic}"}]
        )
        headlines = response["choices"][0]["message"]["content"]
        self.state.headlines = headlines
        return headlines
    @listen(fetch_info)
    def summarize_headlines(self, headlines):
        print(f"summarize_headlines: self.state.topic = {self.state.topic}")
        response = completion(
            model = "gpt-4o-mini",
            messages = [{"role":"user", "content":f"Summarize the headlines briefly:\n{headlines}"}]
        )
        summary = response["choices"][0]["message"]["content"]
        self.state.summary = summary
        return summary

In [ ]:
state = AISummaryState(topic="Agentic AI in Healthcare")
print(f"Topic received in state: {state.topic}")
flow = AISummaryFlow(state=state)
final_output = flow.kickoff()
print(f"Final Output: {final_output}")
print(flow.state)
flow.plot("ai_summary_flow")

Topic received in state: Agentic AI in Healthcare


╭──────────────────────────────────────────────── Flow Execution ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Starting Flow Execution                                                                                        │
│  Name: AISummaryFlow                                                                                            │
│  ID: 9bb1111f-c423-4599-a795-a4cd4640e323                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

 Flow started with ID: 9bb1111f-c423-4599-a795-a4cd4640e323

Output()

fetch_info: self.state.topic = Agentic AI in Health care

AuthenticationError: litellm.AuthenticationError: AuthenticationError: OpenAIException - The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [ ]:
print(f"Final Output: {final_output}")
print("Final State:")
print(flow.state)
flow.plot("ai_summary_flow")